In [85]:
import pandas as pd
import scipy as scp
import numpy as np
import time

from copy import deepcopy
from sklearn.preprocessing import StandardScaler 
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier,plot_tree
from sklearn.model_selection import train_test_split,GridSearchCV,RandomizedSearchCV
from sklearn.metrics import accuracy_score,precision_score,f1_score,recall_score,classification_report,make_scorer
from sklearn.ensemble import RandomForestClassifier
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn.feature_selection import SelectKBest,SelectFpr
from sklearn.pipeline import Pipeline


In [86]:
df_main = pd.read_csv('Ultrasound_1.csv')
df = deepcopy(df_main)
print(df['Pass/Fail'].value_counts()/df.shape[0])
y = df.pop('Pass/Fail').values



-1    0.933418
 1    0.066582
Name: Pass/Fail, dtype: float64


In [87]:
class Processing_Df:
    def __init__(self,df_lcl):
        self.df_cl = df_lcl
        self.lst_fl = []
        self.lst_int = []
        self.lst_o = []
        self.stats_lst = []
        self.lst_d_m_c = []
        
#@@@@@@@@@@@@@@@@@@@@@@@@@$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    def df_num(self):
        #self.df_cl = self.Drop_mis_column()
        for cols in self.df_cl:
            if self.df_cl[cols].dtypes == 'float':
                self.lst_fl.append(cols)
            if self.df_cl[cols].dtypes == 'int':
                self.lst_int.append(cols)
            if self.df_cl[cols].dtypes == 'object':
                self.lst_o.append(cols)    
            
        return (self.lst_fl,self.lst_int,self.lst_o)
#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
    def df_std_removal(self):
        
        df_stats = self.df_cl.describe()
        for i in df_stats.columns:
            if df_stats.loc['std',i] == 0:
                self.stats_lst.append(i)
        self.df_cl.drop(columns=self.stats_lst,inplace=True) 
        return (self.df_cl)
#-------------------------------->>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
    def Drop_mis_column(self):
        dct_df = {cols:(self.df_cl[cols].isnull().sum()/self.df_cl.shape[0])*100 for cols in df.columns}
        for j in dct_df.keys():
            if dct_df[j] > 25.0:
                self.lst_d_m_c.append(j)
        self.df_cl.drop(columns=self.lst_d_m_c,inplace=True)
        return (self.df_cl)
#&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&
    def Standardisation(self):
        tmp = []
        for i in  self.df_cl.columns:
            if self.df_cl[i].max()>3:
                tmp.append(i)
            else:  
                continue
        std_sc = StandardScaler()
        self.df_cl[tmp] = std_sc.fit_transform( self.df_cl[tmp])
        return  self.df_cl
#-----------------------><><><><><><><><<><>-------------------------------------------------------------
    def Mis_Treat(self):
        for i in self.df_cl.columns:
            if self.df_cl[i].isnull().sum() != 0:
                if (self.df_cl[i].skew() > 1) | (self.df_cl[i].skew() < -1):
                    self.df_cl[i].fillna(value=self.df_cl[i].median(),inplace=True)
                elif (self.df_cl[i].skew() < 1) | (self.df_cl[i].skew() > -1):
                     self.df_cl[i].fillna(value=self.df_cl[i].mean(),inplace=True)
        return self.df_cl
#--------------------------->>>>>>>>>>>><<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<------------------------------------------
    def Replacing_outliers(self,tmp):
        df_tmp_zscore = scp.stats.zscore(self.df_cl)
        
        for i in df_tmp_zscore.columns:
            if tmp > 0:
                if (df_tmp_zscore[i]>tmp).any():
                    arr1 = np.where(df_tmp_zscore[i]>tmp)
                    
                    tmp1 = tmp*self.df_cl[i].std() + self.df_cl[i].mean()
                    for j in arr1[0]:
                        self.df_cl.loc[j,i] =  tmp1
            else:
                if (df_tmp_zscore[i]<tmp).any():
                    arr1 = np.where(df_tmp_zscore[i]<tmp)
                   
                    tmp1 = -tmp*self.df_cl[i].std() + self.df_cl[i].mean()
                    for j in arr1[0]:
                        
                        self.df_cl.loc[j,i] =  tmp1
        return self.df_cl
#-------------------------><><><><MAIN><MAIN><><><><><><><>><><><><><><><><---------------------------------------------    def         
    def main(self):
        self.Drop_mis_column()
        self.df_std_removal()
        self.df_cl = self.Mis_Treat()
        self.Replacing_outliers(3.0)
        self.Replacing_outliers(-3.0)
        self.Standardisation()
        return self.df_cl 

In [88]:
df_o = Processing_Df(df)
l_f,l_int,l_o = df_o.df_num()
df_new  =  df_o.main()

#sns.scatterplot(data = df_new,x = range(df.shape[0]),y = )

In [89]:
# df_new = SelectFpr(alpha = 0.02).fit_transform(df_new,y)
# df_new.shape

In [90]:
# for i in df.columns:
#     if df[i].isnull().sum() != 0:
#         if (df[i].skew() > 1) | (df[i].skew() < -1):
#             df[i].fillna(value=df[i].median(),inplace=True)
#         elif (df[i].skew() < 1) | (df[i].skew() > -1):
#             df[i].fillna(value=df[i].mean(),inplace=True)
        

In [91]:
# def Replacing_outliers(df_tmp,tmp):
#     df_tmp_zscore = scp.stats.zscore(df_tmp)
#     for i in df_tmp_zscore.columns:
#         if tmp > 0:
#             if (df_tmp_zscore[i]>tmp).any():
#                 arr1 = np.where(df_tmp_zscore[i]>tmp)
#                 #print((len(arr1[0])/df_tmp.shape[0])*100)
#                 tmp1 = tmp*df_tmp[i].std() + df_tmp[i].mean()
#         else :
#             if (df_tmp_zscore[i]<tmp).any():
#                 arr1 = np.where(df_tmp_zscore[i]<tmp)
#                 tmp1 = -tmp*df_tmp[i].std() + df_tmp[i].mean()
#     for j in arr1[0]:
#         df_tmp.loc[j,i] =  tmp1
#     return df_tmp
       
    
# df_new = Replacing_outliers(df_new,3)
# df_new = Replacing_outliers(df_new,-3)

In [92]:
ad = df_new.corr().abs()
dct_1 = {j : k for j,k in enumerate(df_new.columns)}
upper = ad.where(np.triu(np.ones(ad.shape), k=1).astype(bool))
threshold = 0.9  # Set your threshold value
to_drop = [column for column in upper.columns if any(upper[column] > threshold)]
df_new.drop(columns=to_drop,inplace  = True)
df_new.shape




# for  cols in ad.columns:
#              cmp = np.where((ad[cols] >0.98) | (ad[cols] <-0.98))[0]
    
#              if len(cmp) > 1:
#                 for val in cmp:
#                     lst_corr.append(dct_1[val])
   
#         lst_corr = list(set(lst_corr))
#         self.df_cl.drop(columns = lst_corr,inplace =True)
#         return self.df_cl

(1577, 261)

In [9]:
df_new.shape

(1577, 261)

In [37]:
x = df_new
X_train,X_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=42)


In [38]:
# lr=LogisticRegression(C = 0.1, random_state = 42, solver = 'liblinear')
dt=DecisionTreeClassifier()
rf=RandomForestClassifier()
sel_feat =SelectFpr() 
# gnb=GaussianNB()
# knn = KNeighborsClassifier(n_neighbors=3)
# svm = svm.SVC(kernel='linear')

In [39]:
parameters_grid = [{'feat' : [sel_feat],
                    'feat__alpha' : [0.1,0.08,0.05,0.02],
                     
                    },
                   {'classifier' : [dt],
                   
                    'classifier__max_depth' : [5,15,3],
                   'classifier__criterion' : ['gini' , 'entropy'],
                   #'max_features' : ['log2' , 'sqrt']
                  },
                  
                  { 'classifier' : [rf],
                    'classifier__n_estimators' : [50,500,50],
                    'classifier__max_depth' : [5,15,3],
                   'classifier__criterion' : ['gini' , 'entropy'],
                   #'max_features' : ['log2' , 'sqrt']
                  }]
ppln = Pipeline([('feat',sel_feat),('classifier',dt)])

In [40]:
model_hy_tun = GridSearchCV(ppln, param_grid=parameters_grid, cv = 5, 
                            scoring = {'accuracy': make_scorer(accuracy_score),
                                       're_call': 'recall'},refit='re_call')
model_hy_tun.fit(X_train,y_train)
print(model_hy_tun.best_score_)
print(model_hy_tun.best_params_)
print(model_hy_tun.best_estimator_)


0.23157894736842105
{'feat': SelectFpr(), 'feat__alpha': 0.05}
Pipeline(steps=[('feat', SelectFpr()),
                ('classifier', DecisionTreeClassifier())])


In [26]:
# parameters_grid = { 'n_estimators' : np.arange(50,500,50),
#                     'max_depth' : np.arange(5,15,3),
#                    'criterion' : ['gini' , 'entropy'],
#                    #'max_features' : ['log2' , 'sqrt']
#                   }
# model_hy_tun = RandomizedSearchCV(estimator=rf, param_distributions=parameters_grid, cv = 5, n_iter = 30,
#                             scoring = {'accuracy': make_scorer(accuracy_score),
#                                        're_call': 'recall'},refit='re_call')
# model_hy_tun.fit(X_train,y_train)
# print(model_hy_tun.best_score_)
# print(model_hy_tun.best_params_)


In [27]:
# eval_metric = {}
# for a,b in zip([lr,dt,knn,svm,rm,gnb],["Logistic Regression","Decision Tree","KNN","SVM","Random Forest","Naive Bayes"]):
#     a.fit(X_train,y_train)
#     y_pred=a.predict(X_train)
#     r_c = recall_score(y_train,y_pred)
#     p_c = precision_score(y_train,y_pred)
#     f1 = f1_score(y_train,y_pred)
#     eval_metric[a] = (r_c,p_c,f1)
    
    